In [90]:
import tensorflow as tf
import numpy as np
import os 
import time


In [91]:
path_to_file =  tf.keras.utils.get_file("shakespeare.txt","https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt")


In [4]:
#read
text = open(path_to_file,'rb').read().decode(encoding='utf-8')
print(f'Lengh of text: {len(text)} characters' )


Lengh of text: 1115394 characters


In [5]:
#take a look at the first 250 chars
print(text[:250])


First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [8]:
##unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')


65 unique characters


In [9]:
##vectorize the texts
example_texts = ['abcdefg','xyz']

chars  = tf.strings.unicode_split(example_texts,input_encoding='UTF-8')

In [10]:
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [11]:
##
ids_from_chars = tf.keras.layers.StringLookup(
        vocabulary=list(vocab),mask_token=None)

In [18]:
ids = ids_from_chars(chars)
print(ids)

<tf.RaggedTensor [[40, 41, 42, 43, 44, 45, 46], [63, 64, 65]]>


In [19]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary = ids_from_chars.get_vocabulary(),invert=True,mask_token=None)


In [20]:
chars 

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [21]:
tf.strings.reduce_join(chars,axis=-1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

In [29]:
def text_from_ids(ids):
    return tf.strings.reduce_join(chars_from_ids(ids),axis=-1)

In [23]:
all_ids = ids_from_chars(tf.strings.unicode_split(text,'UTF-8'))
all_ids

<tf.Tensor: shape=(1115394,), dtype=int64, numpy=array([19, 48, 57, ..., 46,  9,  1], dtype=int64)>

In [24]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))
    

F
i
r
s
t
 
C
i
t
i


In [25]:
seq_length = 100

In [26]:
sequences = ids_dataset.batch(seq_length+1,drop_remainder=True)

for seq in sequences.take(1):
    print(chars_from_ids(seq))

tf.Tensor(
[b'F' b'i' b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':'
 b'\n' b'B' b'e' b'f' b'o' b'r' b'e' b' ' b'w' b'e' b' ' b'p' b'r' b'o'
 b'c' b'e' b'e' b'd' b' ' b'a' b'n' b'y' b' ' b'f' b'u' b'r' b't' b'h'
 b'e' b'r' b',' b' ' b'h' b'e' b'a' b'r' b' ' b'm' b'e' b' ' b's' b'p'
 b'e' b'a' b'k' b'.' b'\n' b'\n' b'A' b'l' b'l' b':' b'\n' b'S' b'p' b'e'
 b'a' b'k' b',' b' ' b's' b'p' b'e' b'a' b'k' b'.' b'\n' b'\n' b'F' b'i'
 b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':' b'\n' b'Y'
 b'o' b'u' b' '], shape=(101,), dtype=string)


In [31]:
for seq in sequences.take(5):
    print(text_from_ids(seq).numpy())

b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
b'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
b"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
b"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
b'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


In [41]:
for seq in sequences.take(1):
    print(seq)

tf.Tensor(
[19 48 57 58 59  2 16 48 59 48 65 44 53 11  1 15 44 45 54 57 44  2 62 44
  2 55 57 54 42 44 44 43  2 40 53 64  2 45 60 57 59 47 44 57  7  2 47 44
 40 57  2 52 44  2 58 55 44 40 50  9  1  1 14 51 51 11  1 32 55 44 40 50
  7  2 58 55 44 40 50  9  1  1 19 48 57 58 59  2 16 48 59 48 65 44 53 11
  1 38 54 60  2], shape=(101,), dtype=int64)


In [43]:
def split_input_target(sequences):
    input_text = sequences[:-1]
    target_text = sequences[1:]
    
    return input_text,target_text

In [44]:
split_input_target(list('Tensorflow'))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [45]:
split_input_target(list('akshat'))

(['a', 'k', 's', 'h', 'a'], ['k', 's', 'h', 'a', 't'])

In [46]:
split_input_target(list('chunnu'))

(['c', 'h', 'u', 'n', 'n'], ['h', 'u', 'n', 'n', 'u'])

In [47]:
split_input_target(list('artizence'))

(['a', 'r', 't', 'i', 'z', 'e', 'n', 'c'],
 ['r', 't', 'i', 'z', 'e', 'n', 'c', 'e'])

In [48]:
split_input_target(list('papa'))

(['p', 'a', 'p'], ['a', 'p', 'a'])

In [49]:
split_input_target(list('mummy'))

(['m', 'u', 'm', 'm'], ['u', 'm', 'm', 'y'])

In [50]:
dataset = sequences.map(split_input_target)

In [53]:
dataset.take(1)

<_TakeDataset element_spec=(TensorSpec(shape=(100,), dtype=tf.int64, name=None), TensorSpec(shape=(100,), dtype=tf.int64, name=None))>

In [54]:
dataset = sequences.map(split_input_target)

In [55]:
for input_example,target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target :",text_from_ids(target_example).numpy())

Input : b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target : b'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


In [56]:
## batch size
BATCH_SIZE = 64

BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE,drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))
dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [57]:
## Lenght of the vocabulary in StringLoopup Layer
vocab_size = len(ids_from_chars.get_vocabulary())

## embedding
embedding_dim = 256

#no. of rnn units
rnn_units = 1024


In [75]:
class MyModel(tf.keras.Model):
    def __init__(self , vocab_size,embedding_dim,rnn_units):
        super().__init__(self)
        self.embedding = tf.keras.layers.Embedding(vocab_size,embedding_dim)
        self.gru = tf.keras.layers.GRU(rnn_units,
                                       return_sequences=True,
                                       return_state=True)
        self.dense = tf.keras.layers.Dense(vocab_size)
        
    def call(self,inputs,states=None,return_state=False,training=False):  
        x= inputs
        x = self.embedding(x,training=training)
        if states is None:
            states = self.gru.get_initial_state(x)
        x , states = self.gru(x,initial_state=states, training=training)
        x = self.dense(x, training=training)
        
        if return_state:
            return x, states
        else:
            return x
        
    

In [76]:
model = MyModel(vocab_size=vocab_size,embedding_dim=embedding_dim,rnn_units=rnn_units)

In [77]:
## models

for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape,"# (batch_size,sequences_length , vocab_size) ")

(64, 100, 66) # (batch_size,sequences_length , vocab_size) 


In [80]:
dataset.take(1)

<_TakeDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [81]:
model.summary()

Model: "my_model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     multiple                  16896     
                                                                 
 gru_4 (GRU)                 multiple                  3938304   
                                                                 
 dense_4 (Dense)             multiple                  67650     
                                                                 
Total params: 4,022,850
Trainable params: 4,022,850
Non-trainable params: 0
_________________________________________________________________


In [83]:
sampled_indices = tf.random.categorical(example_batch_predictions[0],num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [84]:
sampled_indices

array([ 3, 15, 46,  3, 48, 18, 63, 19, 22, 47, 35, 48, 23, 16, 41, 38, 32,
       57,  4, 26, 54, 36, 53, 17, 37, 65, 39, 38, 58, 18, 42, 10, 36, 28,
       48,  2, 27, 19, 24, 65,  1,  7, 52, 65, 47, 48,  7, 49, 27, 40, 62,
       20, 56, 42, 10, 22, 48, 51, 25, 42, 17, 24, 57, 36, 11, 43, 10, 41,
       62, 25, 34, 31,  0, 17, 17, 64, 20,  4, 53, 31, 14, 36, 64, 60, 14,
        0, 29, 32, 27, 60, 43, 56, 59, 50, 16, 30, 53, 17, 31, 27],
      dtype=int64)

In [85]:
print("Input: \n",  text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions: \n",text_from_ids(sampled_indices).numpy())

Input: 
 b"t be not a bawd's house,\nit is pity of her life, for it is a naughty house.\n\nESCALUS:\nHow dost thou "

Next Char Predictions: 
 b'!Bg!iExFIhViJCbYSr$MoWnDXzZYsEc3WOi NFKz\n,mzhi,jNawGqc3IilLcDKrW:d3bwLUR[UNK]DDyG$nRAWyuA[UNK]PSNudqtkCQnDRN'


In [86]:
##training the model

In [89]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)
##example

In [92]:
example_batch_mean_loss = loss(target_example_batch,example_batch_predictions)
print("Predictions shape:",example_batch_predictions.shape,"# (batch_size,sequence_length,vocab_suze) ")
print("Mean loss:   ", example_batch_mean_loss)


Predictions shape: (64, 100, 66) # (batch_size,sequence_length,vocab_suze) 
Mean loss:    tf.Tensor(4.187724, shape=(), dtype=float32)


In [93]:
tf.exp(example_batch_mean_loss).numpy()

65.8727

In [94]:
model.compile(optimizer='adam',loss=loss)

In [95]:
## Directory
checkpoint_dir = './training_checkpoints'
##name of the checkpoint files
checkpoint_prefix  = os.path.join(checkpoint_dir,"ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath = checkpoint_prefix,
    save_weights_only = True
)

In [96]:
EPOCHS = 20

In [ ]:
## Training the model

In [ ]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])


Epoch 1/20
172/172 [==============================] - 600s 3s/step - loss: 2.7385
Epoch 2/20
150/172 [=========================>....] - ETA: 1:16 - loss: 2.0191

## Training the model

In [ ]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states
        

In [ ]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)


In [ ]:
##
start = time.time()
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)


In [ ]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result, '\n\n' + '_'*80)
print('\nRun time:', end - start)

In [ ]:
##export generator
tf.saved_model.save(one_step_model, 'one_step')
one_step_reloaded = tf.saved_model.load('one_step')

In [ ]:
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(100):
  next_char, states = one_step_reloaded.generate_one_step(next_char, states=states)
  result.append(next_char)

print(tf.strings.join(result)[0].numpy().decode("utf-8"))